<img src="../img/banner.png" width="500" align="center">

# Procesamiento de lenguaje natural
## Rule-Based BOT con TF-IDF

Preparado por: Alan Churichi

En este notebook se realizará un Rule-Based BOT utilizando la librería SpaCy para relaizar la tokenización y lematización. Luego se usarán los conceptos de TF-IDF y Similitud del Coseno vistos en la primera clase para la creacion del BOT.

In [1]:
%load_ext lab_black

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import bs4 as bs
import html
import re
import spacy
import string
import urllib.request

Cargamos el diccionario 'en_core_web_sm' y aumentamos la longitud máxima para poder cargar textos de mayor tamaño.

In [2]:
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 2000000

En este caso utilizaremos el texto del libro "Guía del autoestopista galáctico" para entrenar nuestro chatbot. El mismo se puede encontrar en el siguiente [link]("https://archive.org/stream/TheultimateHitchhikersGuide/The%20Hitchhiker%27s%20Guide%20To%20The%20Galaxy_djvu.txt").

Luego extraemos la parte donde se encuentra el texto a procesar.

In [3]:
def get_html_article(url):
    raw_html = urllib.request.urlopen(url)
    raw_html = raw_html.read()
    return bs.BeautifulSoup(raw_html, "html.parser")


def extract_text_from_html(html, startDelimiter, endDelimiter):
    txt = str(html)
    start = txt.find(startDelimiter) + len(startDelimiter)
    end = txt.find(endDelimiter)
    return txt[start:end]


html_article = get_html_article(
    "https://archive.org/stream/TheultimateHitchhikersGuide/The%20Hitchhiker%27s%20Guide%20To%20The%20Galaxy_djvu.txt"
)
article = extract_text_from_html(html_article, "<pre>", "</pre>")

Preprocesamos el articulo removiendo caracteres especiales y convertimos los símbolos html.

In [4]:
def preprocess_article(article):
    # Remover caracteres especiales
    filtered_string = re.sub(r"\[[0-9]*\]", " ", article)
    filtered_string = re.sub(r"\s+", " ", filtered_string)
    # Convertir simbolos especiales html. Ejemplo: &amp; -> &
    filtered_string = html.unescape(filtered_string)
    return filtered_string.lower()


article = preprocess_article(article)

Utilizamos SpaCy para dividir el texto por oraciones y generamos el Corpus.

In [5]:
doc = nlp(article)

corpus = [sent.text.strip() for sent in doc.sents]
corpus[:5]

["douglas adams the ultimate hitchhiker's guide complete & unabridged contents: introduction: a guide to the guide the hitchhiker's guide to the galaxy the restaurant at the end of the universe life, the universe and everything so long, and thanks for all the fish young zaphod plays it safe mostly harmless footnotes introduction: a guide to the guide some unhelpful remarks from the author the history of the hitchhiker's guide to the galaxy is now so complicated that every time i tell it i contradict myself, and whenever i do get it right i'm misquoted.",
 'so the publication of this omnibus edition seemed like a good opportunity to set the record straight - or at least firmly crooked.',
 "anything that is put down wrong here is, as far as i'm concerned, wrong for good.",
 'the idea for the title first cropped up while i was lying drunk in a field in innsbruck, austria, in 1971.',
 'not particularly drunk, just the sort of drunk you get when you have a couple of stiff gossers after not 

Se crea la función 'get_processed_text' para poder tokenizar y lematizar el texto de entrada.

In [6]:
def get_processed_text(text):
    doc = nlp(text)
    lemma_list = [token.lemma_.lower() for token in doc]
    filtered_sentence = [w for w in lemma_list if w not in string.punctuation]
    return filtered_sentence


get_processed_text("Tokenizing this example!")

['tokenize', 'this', 'example']

Utilizamos la clase 'TfidfVectorizer' de Sklearn para obtener una matriz TF-IDF del corpus y luego usamos 'cosine_similarity' para encontrar el documento que guarda mas similaridad con el texto ingresado como input. Este documento será la respuesta que se mostrará al usuario.

In [7]:
def generate_response(user_input, corpus):
    response = ""
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    word_vectorizer = TfidfVectorizer(
        tokenizer=get_processed_text,  # Obtener los tokens lematizados
        stop_words=get_processed_text(
            " ".join(ENGLISH_STOP_WORDS)
        ),  # Quitar las "stop words" del ingles. Preprocesamos para obtener tokens lematizados correctos
    )

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    response = (
        "I am sorry, I could not understand you"
        if vector_matched == 0
        else corpus[similar_sentence_number]
    )

    corpus.remove(user_input)
    return response

Probamos el bot ingresando diferentes preguntas y viendo las respuestas que nos devuelve.

In [8]:
def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

In [10]:
questions = [
    "Do you know who is Arthur Dent?",
    "Can you tell me a good story?",
    "Will our planet be destroyed?",
    "What means being alive?",
]

for question in questions:
    answer = generate_response(question.lower(), corpus)
    print(f"Q: {question}")
    print(f"A: {answer}\n")

Q: Do you know who is Arthur Dent?
A: "you are arthur dent?

Q: Can you tell me a good story?
A: "now you tell me a story."

Q: Will our planet be destroyed?
A: it was destroyed!"

Q: What means being alive?
A: we are still alive, and we are about to lose our lives."



Como se puede ver las respuestas guardan cierto sentido con la pregunta realizada pero son cortas y no aportan mucha información. Esto posiblemente se deba a que el texto contiene muchas sentencias cortas que a la hora de compararse utilizando la similitud del coseno tienen mucha relación con la pregunta hecha y se eligen como respuesta.

Tambien hay que destacar que el chatbot no es capaz de generar texto, simplemente usa las sentencias en el corpus con más similitud, por eso en algunos casos las respuestas pueden no ser del todo precisas.